In [14]:
import numpy as np
import numpy.random as npr
import pandas as pd
import scanpy as sc
import torch
torch.manual_seed(0);

In [15]:
hpa_mapping = pd.Series({
    'epithelial cell of proximal tubule':'proximal tubule',
    'fibroblast':None,
    'glomerular visceral epithelial cell':'glomerulus',
    'kidney capillary endothelial cell':'glomerulus',
    'kidney connecting tubule epithelial cell':'distal tubule',
    'kidney distal convoluted tubule epithelial cell':'distal tubule',
    'kidney loop of Henle thick ascending limb epithelial cell':'distal tubule',
    'leukocyte':None,
    'mesangial cell':'glomerulus',
    'parietal epithelial cell':'glomerulus',
    'renal alpha-intercalated cell':'collecting duct',
    'renal beta-intercalated cell':'collecting duct',
    'renal principal cell':'collecting duct'
},name='region')

In [16]:
hpa_markers = pd.read_csv('./data/hpa_markers.csv',comment='#')
gene_symbols = pd.read_table('./data/gene_symbols.tsv').iloc[:,[0,-1]]
hpa_markers = hpa_markers.merge(gene_symbols,
                                left_on='gene',right_on='Approved symbol',how='inner')
hpa_markers = hpa_markers.set_index(hpa_markers.columns[-1])[['region','gene']]
hpa_markers

,region,gene
Ensembl ID(supplied by Ensembl),,
ENSG00000128567,glomerulus,PODXL
ENSG00000158457,glomerulus,TSPAN33
ENSG00000113578,glomerulus,FGF1
ENSG00000116218,glomerulus,NPHS2
ENSG00000198743,glomerulus,SLC5A3
...,...,...
ENSG00000105707,collecting duct,HPN
ENSG00000214128,collecting duct,TMEM213
ENSG00000100362,collecting duct,PVALB


In [17]:
adata = sc.read('./data/adata.h5')
adata

AnnData object with n_obs × n_vars = 10164 × 128
    obs: 'Batch', 'Slide', 'Well', 'Tissue', 'Gene name', 'Gene', 'UniProt', 'Antibody', 'nTPM', 'Staining', 'Sex', 'Age', 'Patient', 'URL', 'Level', 'Reliability', 'epithelial cell of proximal tubule', 'fibroblast', 'glomerular visceral epithelial cell', 'kidney capillary endothelial cell', 'kidney connecting tubule epithelial cell', 'kidney distal convoluted tubule epithelial cell', 'kidney loop of Henle thick ascending limb epithelial cell', 'leukocyte', 'mesangial cell', 'parietal epithelial cell', 'renal alpha-intercalated cell', 'renal beta-intercalated cell', 'renal principal cell', 'rna_cell_type', 'rna_specificity', 'leiden'
    uns: 'Patient_colors', 'leiden', 'leiden_colors', 'neighbors', 'rna_cell_type_colors', 'umap'
    obsm: 'X_umap'
    obsp: 'connectivities', 'distances'

In [33]:
train_genes = set(adata.obs.dropna()['Gene'])
test_genes = train_genes & set(hpa_markers.index)
train_genes = train_genes - set(hpa_markers['gene'])

len(train_genes), len(test_genes)

(2106, 88)

In [32]:
train_mask = adata.obs['Gene'].isin(train_genes)
test_mask = adata.obs['Gene'].isin(test_genes)

In [ ]:
# from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
# from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score
# from scipy.special import softmax
# from sklearn.model_selection import GroupKFold

# from src.classifier import SoftmaxRegression

In [ ]:
# X_train = adata[train_mask].to_df()
# Y_train = adata[train_mask].obs[rna.columns].copy()

# X_test = adata_test.to_df()
# Y_test = adata_test.obs[rna.columns].copy()
# clf = SoftmaxRegression(max_iters=1000, lr=0.01, verbose=True)

# labels_train = Y_train.idxmax(1)
# w_train = (1./labels_train.value_counts())[labels_train]
# groups_train = adata_train[train_idx].obs['Gene']

# shape = Y_train.shape
# columns = Y_train.columns
# # Y_train = labels_train.copy()

# npr.seed(0)
# idx = npr.permutation(len(X_train)) # shuffle
# Y_pred_train = np.zeros(shape)
# Y_pred_train[idx] = cross_val_predict(
#     clf,
#     X_train.iloc[idx],
#     Y_train.iloc[idx], 
#     groups=groups_train.iloc[idx],
#     method='predict_proba',
#     cv=GroupKFold(n_splits=5)
# )
# Y_pred_train /= Y_pred_train.sum(1,keepdims=True)
# Y_pred_train = pd.DataFrame(Y_pred_train,columns=columns,index=X_train.index)

# labels_pred_train = Y_pred_train.idxmax(1)

# clf.fit(X_train,Y_train)#,sample_weight=w_train)
# Y_pred_test = clf.predict_proba(X_test)
# # Y_pred_test = clf.predict(X_test)
# Y_pred_test /= Y_pred_test.sum(1,keepdims=True)
# Y_pred_test = pd.DataFrame(Y_pred_test,columns=columns,index=X_test.index)

# print(balanced_accuracy_score(labels_train, labels_pred_train))

# plt.figure(figsize=(8,8))
# ls = [c[:20] for c in rna.columns]
# conf_mtx = pd.DataFrame(confusion_matrix(labels_train,labels_pred_train),index=ls,columns=ls)

# g = sns.heatmap(np.log(conf_mtx+1),square=True,annot=conf_mtx,cmap='Blues',fmt='d')